In [18]:
#load environment variables
from dotenv import load_dotenv  
load_dotenv()

True

In [19]:
#Create API client
from anthropic import Anthropic
client = Anthropic()
model = "claude-sonnet-4-0"

In [20]:
#helper method to add user messages
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)

#helper method to add assistant messages
def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)

#Make a request to the API
def chat(messages,system=None,temperature=1.0,stop_sequences=[]):
    params ={
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences
    }

    if system:
        params["system"] = system

    response = client.messages.create(**params)
    return response.content[0].text  # Return the text of the first message in the response

#Usage of system Prompts

In [21]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""
    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages,stop_sequences=["```"])
    return json.loads(text)

dataset = generate_dataset()

dataset

# Save the dataset to a file
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [22]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
Please solve the following task:

{test_case['task']}
"""
    message = []
    add_user_message(message, prompt)
    output = chat(message)
    return output


In [23]:
def grade_by_model(test_case, output):
    # Create evaluation prompt
    eval_prompt = """
    You are an expert code reviewer. Evaluate this AI-generated solution.
    
    Task: {task}
    Solution: {solution}
    
    Provide your evaluation as a structured JSON object with:
    - "strengths": An array of 1-3 key strengths
    - "weaknesses": An array of 1-3 key areas for improvement  
    - "reasoning": A concise explanation of your assessment
    - "score": A number between 1-10
    """
    
    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")

    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [24]:
def run_test_case(test_case):
    """calls the run_prompt function with the test case and grades the result"""
    output = run_prompt(test_case)
    # Here you would implement the grading logic based on the output
    
    # Grade the output
    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]
    
    return {
        "output": output, 
        "test_case": test_case, 
        "score": score,
        "reasoning": reasoning
    }

In [25]:
from statistics import mean

def run_eval(dataset):
    """Runs the evaluation on the dataset and returns the results"""
    results = []
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)
    
    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results


In [26]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)


Average score: 4.666666666666667


In [27]:
print(json.dumps(results, indent=2))

[
  {
    "output": "Here's a Python function that extracts the bucket name from an S3 bucket ARN:\n\n```python\ndef extract_bucket_name_from_arn(arn):\n    \"\"\"\n    Extract the bucket name from an S3 bucket ARN.\n    \n    Args:\n        arn (str): S3 bucket ARN in the format 'arn:aws:s3:::bucket-name'\n    \n    Returns:\n        str: The bucket name extracted from the ARN\n    \n    Raises:\n        ValueError: If the ARN format is invalid\n    \"\"\"\n    if not isinstance(arn, str):\n        raise ValueError(\"ARN must be a string\")\n    \n    if not arn:\n        raise ValueError(\"ARN cannot be empty\")\n    \n    # Split the ARN by colons\n    arn_parts = arn.split(':')\n    \n    # Validate ARN format\n    if len(arn_parts) != 6:\n        raise ValueError(\"Invalid ARN format. Expected format: 'arn:aws:s3:::bucket-name'\")\n    \n    if arn_parts[0] != 'arn':\n        raise ValueError(\"ARN must start with 'arn'\")\n    \n    if arn_parts[1] != 'aws':\n        raise ValueE